# Семестровый проект.
# Рекомендательная система для сериалов
### Сайт — myshows.me

#### Шадрин Сергей, Слепцов Александр, Данько Артем

### Задачи и цели

Цель проекта — рекомендательная система для сериалов, то есть предсказание рейтинга непросмотренного сериала для какого либо пользователя.

Поставленные задачи:
- Сбор и подготовка данных;
- Выбор модели и метрики для оценки качества предсказания.

### Сбор данных

Для начала с сайта myshows.me, посвященного сериалам, был собран список из 150 тысяч пользователей. На странице пользователя отображаются его оценки, выставленные сериалам, причем сериалы делятся на следующие категории:
  - "смотрит" (просматриваемые в данный момент),
  - "будет смотреть" (сюда также включены еще невышедшие сериалы),
  - "перестал" (сериалы, просмотр которых прекращен),
  - "полностью посмотрел" (соответственно, просмотренные). 

Далее для каждого из 150 тысяч пользователей был получен список отмеченных им сериалов. Получилось 13096257 записей. Полученные данные состоят из трех столбцов: порядковый номер обработанной ссылки пользователя, ID сериала на сайте, выставленная этому сериалу оценка. 

Значения последнего столбца находится в диапазоне от 0 до 5, где 0 означает, что оценка данному сериалу не была выставлена. Данные с невыставленной оценкой не несут особой ценности, поэтому убираем их. В итоге мы получили 7112074 записей.

Страница с пользователями myshows.me/community/users/

<img src="rec_sys_images/users_page.png">

Страница пользователя myshows.me/username

<img src="rec_sys_images/user_page.png">

```python
site = 'https://myshows.me/community/users/?page={}'
max_num_page = 5000

def extract_users_from_page(site_num):
    soup = BeautifulSoup(requests.get(site.format(site_num)).text, 'html.parser')
    res = []
    
    links = soup.find_all('a', class_='userBlock linkBlock')
    for link in links:
        res.append(link.attrs['href'])
    
    return res

def extract_grades_from_user_page(user_url):
    soup = BeautifulSoup(requests.get(user_url).text, 'html.parser')
    res = {}
    
    tabs = soup.find_all('div', class_='tabs_cont')
    
    for completed_tab in tabs:
        all_grades = completed_tab.find_all('tr')
        for grade in all_grades[:-1]:
            grade_parts = grade.find_all('td')
            id_s = grade_parts[0].find('a').attrs['href'].split('/')[-2]
            grade_value = grade_parts[1].find('span').attrs['class'][1][1]
            res[id_s] = grade_value
    
    return res
```

In [56]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

from scipy.sparse.linalg import svds
from scipy.sparse import coo_matrix
from sklearn.preprocessing import LabelEncoder

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

from scipy.spatial.distance import cosine
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from scipy.spatial.distance import correlation
from sklearn.metrics import pairwise_distances
import math

from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors

In [16]:
df_ratings = pd.read_csv('../data/ratings_without_zeros.csv',
                         header=None,
                         names=['user_id', 'show_id', 'rating_val'],
                         sep=' ')

df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7112074 entries, 0 to 7112073
Data columns (total 3 columns):
user_id       int64
show_id       int64
rating_val    int64
dtypes: int64(3)
memory usage: 162.8 MB


In [28]:
print(df_ratings.shape)
df_ratings.head(10)

(7112074, 3)


,user_id,show_id,rating_val
0,1,55877,4
1,1,55672,4
2,1,41907,4
3,1,34737,5
4,1,44997,5
5,1,50726,4
6,1,48017,4
7,1,42707,3
8,1,26428,5
9,1,331,4


### Подготовка данных

Разобьем данные на обучение и контроль в пропорции 95/5.

В полном объеме датасет не будем использовать. Возьмем первые 100000 строк, из них на обучение — 95000, на контроль — 5000.

В качестве метрики для качества будет использовать MSE:
    $$ MSE = \sum_{(u, m)}(r_{um} - y_{um})^{2} $$

Здесь $r_{um}$ — предсказанный рейтинг фильма $m$ для пользователя $u$, $y_{um}$ — рейтинг, который в действительности проставил пользователь.

In [22]:
train, test = df_ratings.iloc[:95000], df_ratings.iloc[95000:100000]

In [23]:
# Преобразуем train в матрицу
train_table = train.pivot(index='user_id', columns='show_id', values='rating_val')
print(train_table.shape)
train_table.head()

(714, 8862)


show_id,1,2,3,4,5,6,7,8,9,10,...,59296,59297,59307,59314,59331,59347,59352,59364,59411,59414
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5.0,5.0,3.0,NaN,NaN,3.0,3.0,4.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5.0,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Baseline алгоритм

С помощью mse этого алгоритма будем оценивать последующие модели.
Будем считать оценку пользователя $u$ для товара $i$ как $b_{ui}$:

* $b_{ui} = \mu + b_u + b_i$,
* $b_{u} = \frac{1}{|I_u|+\alpha}\sum_{i\in I_u}(R_{ui} - \mu)$
* $b_{i} = \frac{1}{|U_i|+\beta}\sum_{u\in U_i}(R_{ui} - b_u - \mu)$

Интерпретация:
* $b_u$ — насколько выше (ниже) среднего пользователь оценивает товары;
* $b_i$ — насколько выше (ниже) среднего оценивается товар;
* $\mu$ — просто общий средний рейтинг;
* $U_i$ — множество пользователей, оценивших товар $i$;
* $I_u$ — множество товаров, оценненных пользователем $u$;
* $\alpha$, $\beta$ — коэффиценты для сглаживания.

In [57]:
matr = train_table.values

mu = matr[~np.isnan(matr)].mean()
alpha = 0.1
beta = 0.1

pred = np.zeros(len(test))

for i in range(len(pred)):
    uid, sid, _ = list(test.values[i, :])

    ind1 = (train_table.index.values == uid)
    ind2 = (train_table.columns.values == sid)
    
    b_u = 0
    row = matr[ind1, :]
    row_len = len(row[~np.isnan(row)])
    row_sum = row[~np.isnan(row)].sum()
    b_u = (row_sum - row_len * mu) / (row_len + alpha)
    
    b_i = 0
    col = matr[:, ind2]
    col_len = len(col[~np.isnan(col)])
    col_sum = col[~np.isnan(col)].sum()
    b_i = (col_sum - col_len * (mu + b_u)) / (col_len + beta)
    
    pred[i] = mu + b_u + b_i

print(mean_squared_error(test.values[:, 2], pred))

1.3017415001077826


### User-based CF

Так как количество юзеров у нас меньше, чем объектов, будем использовать User-based алгоритм:

* Посчитаем сходство между пользователями $s \in \mathbb{R}^{U \times U}$
* Для целевого пользователя $u$ найти похожих пользователей $N(u)$
$$ \hat{R}_{ui} = \bar{R}_u + \frac{\sum_{v \in N(u)} s_{uv}(R_{vi} - \bar{R}_v)}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$

* $\bar{R}_u$ - поправка на писсимизм\оптимизм пользователей

В качестве функции сходства будем использовать корреляцию Пирсона. Похожих пользователей будем рассматривать, только если в пересечении сериалов, которые они посмотрели, больше 5 элементов. Для получения оценки будем брать только 20 ближайших юзеров.

In [59]:
def my_similarity(u, v):
    ind = ((u != 0) & (v != 0))
    
    if len(u[ind]) <= 5:
        return 100
    
    return correlation(u[ind], v[ind])

In [61]:
# для pairwise_distances требуется, чтобы не было NaN'ов
# поэтому заменяем NaN на 0
res_corr = pairwise_distances(train_table.fillna(0).values, metric=my_similarity)
np.fill_diagonal(res_corr, 100)
res_corr

/home/steady/anaconda3/lib/python3.7/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


KeyboardInterrupt: 

In [ ]:
# with open('./test.txt', 'r') as fin, open('./res_f.csv', 'w') as fout:
#     fout.write('Id,Score\n')
    
#     num_row = 1
    
#     for line in fin:
#         uid, mid = list(map(int, line.split()))
# #         print(uid, mid)
#         inds = np.argpartition(res_corr[uid - 1, :], 20)[:20]
        
#         summ = 0
#         znam = 0

#         for i in inds:
#             summ += (ratings_matrix[i, mid - 1] - ratings_matrix[i, :].mean()) * res_corr[uid - 1, i]
# #             summ += (ratings_matrix[i, mid - 1] - ratings_matrix[i, :].mean())
#             znam += abs(res_corr[uid - 1, i])

# #         t = ratings_matrix[uid - 1, :].mean() + summ / znam
#         t = ratings_matrix[uid - 1, :].mean() + summ / znam
        
#         if math.isnan(t):
#             t = 0
            
#         if t < 0:
#             t = 0
        
#         fout.write('{},{}\n'.format(num_row, round(t, 1)))
#         num_row += 1